In [ ]:
!rm -rf /content/sumanth_setup

%load_ext autoreload
%autoreload 2

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install datasets
!pip -q install transformers
!pip -q install peft
!pip -q install -U bitsandbytes
!pip -q install rouge


import numpy as np
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS6220 Folder #add direct access to the folder if you get an error in this cell
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device="cuda"
!pip install huggingface-cli

!huggingface-cli login

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import numpy as np
from datasets import load_dataset, concatenate_datasets

from transformers import (
    AutoTokenizer,
    AutoModelWithLMHead,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
)
from nltk.translate.bleu_score import sentence_bleu
import os


from datasets import load_dataset

import torch
from torch.utils.data import Dataset

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

import pickle

import time
from tqdm import tqdm

import copy

import random

from bs4 import BeautifulSoup


In [ ]:
!mkdir /content/sumanth_setup
!cp './helpers/' -r '/content/sumanth_setup'

In [ ]:
os.chdir('/content/sumanth_setup')

## Load MBPP Dataset


In [ ]:
# Configure dataset

class CustomDataset(Dataset):

    def __init__(self, data, problem_nums): #initializes the dataset with the folder path, delimiter, and chunk size
        # problem_nums_strings = [str(num).zfill(5) for num in problem_nums]
        self.data = []

        for problem_num in problem_nums:
            problem_metadata = "\"\"\"\n" + data[problem_num]['prompt'] + "\n\"\"\"\n"

            self.data.append(problem_metadata + data[problem_num]['code'])

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Configer hyperparameters for training

num_clients = 2
max_length = 150
batch_size = 8
conduct_logging = True
epochs = 1000
learning_rate = 4e-4
t_max = epochs // 10

training_args = {
    'clients': num_clients,
    'MAX_LENGTH': max_length,
    'BATCH_SIZE': batch_size,
    'conduct_logging': conduct_logging,
    'EPOCHS': epochs,
    'lr': learning_rate,
    't_max': t_max,
}

In [ ]:
# Configure hyperparameters for model

model_name = 'Deci/DeciCoder-1b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
# target_modules = ["gate_proj", "up_proj", "down_proj"]
lora_config = LoraConfig(
    r=8,
    target_modules=target_modules,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model_args = {
    'model_name': model_name,
    'tokenizer': tokenizer,
    'datasets_list': None,
    'quant_config': quant_config,
    'lora_config': lora_config
}

In [ ]:
SAVE_PATH = "./checkpoints/deci_model_mbpp"
LOSSES_PATH = "./logs/losses/deci_model_mbpp_loss"
TIMES_PATH = "./logs/times/deci_model_mbpp_elapsed_time"

In [ ]:
# !mkdir ./checkpoints
# !mkdir ./logs
# !mkdir ./logs/losses
# !mkdir ./logs/times

In [ ]:
mbpp_dataset = load_dataset('mbpp', 'sanitized')
mbpp_dataset = mbpp_dataset['train']

In [ ]:
num_problems_per_client = 8

# Generate num_problems_per_client * num_clients random idxs within dataset
np.random.seed(0)
selection = np.random.choice(np.arange(len(mbpp_dataset)), size=num_clients * num_problems_per_client, replace=False).tolist()

custom_dataset = []
for i in range(num_clients):
    custom_dataset.append(CustomDataset(mbpp_dataset, selection[i * num_problems_per_client: (i + 1) * num_problems_per_client]))

model_args['datasets_list'] = custom_dataset

In [ ]:
# import helpers
# import importlib
# importlib.reload(helpers)

# from helpers.train_ronit_modifs import generic_training_runner

generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_args,
    training_args,
)

Device Type: cuda
Loading Data and Producing DataLoader objects
Finished loading data
Beginning training iteration for client 0
The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Loaded global model.
Number of batches: 1
Starting training


0it [00:00, ?it/s]

Step: 0, Loss: 5.4777


OutOfMemoryError: ignored

In [ ]:
!zip -r /content/trained_decicoder_mbpp.zip content/sumanth_setup

Upload only zipped sumanth_setup folder

In [ ]:
# If directory already exists, skip this
!unzip /content/sumanth_setup.zip -d /content/sumanth_setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

In [ ]:
import helpers
import importlib
importlib.reload(helpers)

from helpers.evaluation import generate_batch_completions

test_prompt = mbpp_dataset[0]['prompt']

client_model = AutoModelForCausalLM.from_pretrained(
    SAVE_PATH + f"_{0}", quantization_config=quant_config, device_map={"": 0}
)
client_model.gradient_checkpointing_enable()

client_model = prepare_model_for_kbit_training(client_model)
client_model = get_peft_model(client_model, lora_config).to(device)


The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
print(mbpp_dataset[selection[4]]['prompt'])

formatted_prompt = "\"\"\"\n" + mbpp_dataset[selection[4]]['prompt'] + "\n\"\"\"\n"

output = generate_batch_completions([formatted_prompt], client_model, tokenizer)
print(output[0])


Write a function to convert tuple string to integer tuple.
Device Type: cuda
"""
Write a function to convert tuple string to integer tuple.
"""

def tuple_to_int(tuple_str):
    """
    Convert tuple string to integer tuple.
    """
    return tuple(int(x) for x in tuple_str)




In [ ]:
global_model = AutoModelForCausalLM.from_pretrained(
    SAVE_PATH + f"_{0}", quantization_config=quant_config, device_map={"": 0}
)
global_model.gradient_checkpointing_enable()

global_model = prepare_model_for_kbit_training(global_model)
global_model = get_peft_model(global_model, lora_config).to(device)



The repository for Deci/DeciCoder-1b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciCoder-1b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
print(mbpp_dataset[selection[4]]['prompt'])

formatted_prompt = "\"\"\"\n" + mbpp_dataset[selection[4]]['prompt'] + "\n\"\"\"\n"

output = generate_batch_completions([formatted_prompt], global_model, tokenizer)
print(output[0])

Write a function to convert tuple string to integer tuple.
Device Type: cuda
"""
Write a function to convert tuple string to integer tuple.
"""

def tuple_to_int(tuple_str):
    """
    Convert tuple string to integer tuple.
    """
    return tuple(int(x) for x in tuple_str)


